In [1]:
import requests
import json
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from database_utils import Database
import pandas as pd
import time
import datetime
import random
import sys

# 根据关键词查出所有视频的相关信息

In [ ]:
all_follows = set()



In [ ]:
for i in range(1, 6):
    s = requests.get('https://api.bilibili.com/x/relation/followings?vmid=20165629&pn='+ str(i) +'&ps=52').text
    users = json.loads(s)['data']['list']
    all_follows = all_follows | set([u['uname'] for u in users])

In [ ]:
len(all_follows)

In [ ]:
keyword="鸿蒙系统"
res=[]

In [ ]:
# 按url获取当前页的视频列表
def get_video_list(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    all_videos = soup.find_all(name='ul',attrs={"class":"video-contain clearfix"})[0]
    all_videos = all_videos.find_all(name='div', attrs={"class":"info"})
    return all_videos
# 从url中分离出up主id
def get_up_id(url):
    return url[url.index("com/") + 4: url.index("?")]
# 获取视频中的信息
def get_vider_data(source, res):
    for i, every_video in enumerate(source):
        data = {}
        data['av_id'] = every_video.find_all(name='span', attrs={"class":"type avid"})[0].text[2:]
        data['title'] = every_video.a['title']

        tags = every_video.find_all(name='div', attrs={"class":"tags"})[0]
        for j, s in enumerate(tags):
            data[s['title']] = s.text.strip()
            if j == 3:
                data['up_id'] = get_up_id(s.a['href'])
        res.append(data)

In [ ]:
for page in range(1, 100):
    url = r'https://search.bilibili.com/all?keyword=' + keyword +'&page=' + str(page)
    video_list = get_video_list(url)
    if len(video_list) == 0:
        print("共%d页"% (page - 1))
        break
    else:
        get_vider_data(video_list, res)   

video_info = pd.DataFrame(data=res)
# video_info.to_csv("video_info.csv")

In [ ]:
video_info

# 爬取所有视频的弹幕

In [ ]:
barrages = []
av_id_list = video_info[video_info['弹幕'] != '0']['av_id'].tolist()

In [ ]:
def get_cid(avid):
    source = requests.get("https://api.bilibili.com/x/web-interface/view?aid=" + str(avid)).text
    source = json.loads(source)
    cid = source['data']['cid']
    return cid

In [ ]:
cols_name = ['video_time', 'barrage_type', 'font_size', 'font_color',
             'time', 'barrage_pool_type', 'sender_id', 'barrage_id']
def get_barrages(avid):
    curr_list = []
    cid = get_cid(avid)
    source = requests.get("https://comment.bilibili.com/"+ str(cid) +".xml")
    source.encoding = 'utf-8'
    source = BeautifulSoup(source.text, 'html.parser')
    source = source.find_all(name='d')
    for line in source:
        data = {'av_id': avid, 'content': line.text}
        for i, val in enumerate(line['p'].split(',')):
            data[cols_name[i]] = val
#     source = json.loads(source)
        curr_list.append(data)
    return curr_list

In [ ]:
for i, avid in enumerate(av_id_list):
    barrages += get_barrages(avid)

In [ ]:
barrages_csv = pd.DataFrame(data=barrages)

In [15]:
barrages_csv.shape

(11560, 10)

# 获取弹幕中的所有u_id

In [ ]:
def get_uid(x):
    res = database_util.get_uid(x)
    return res


In [ ]:
database_util = Database()
barrages_csv['u_id'] = barrages_csv['sender_id'].apply(lambda x: get_uid(x))
database_util.close()

In [ ]:
barrages_csv.to_csv("barrage_csv.csv", index=False)

# 获取弹幕中所有用户的信息

In [2]:
barrages_csv = pd.read_csv("barrage_csv.csv")

In [3]:
barrages_csv = barrages_csv[barrages_csv['u_id'].notnull()]
barrages_csv['u_id'] = barrages_csv['u_id'].astype(int).astype(str)

In [25]:
def LoadUserAgents(uafile):
    uas = []
    with open(uafile, 'rb') as uaf:
        for ua in uaf.readlines():
            if ua:
                uas.append(ua.strip()[:-1])
    random.shuffle(uas)
    return uas

uas = LoadUserAgents("user_agents.txt")


head = {
    'User-Agent': random.choice(uas),
    'Connection': 'close',
}

def get_user_info(uid):
    data = {'u_id': uid}
    requests.adapters.DEFAULT_RETRIES =5
    source = requests.get("https://api.bilibili.com/x/space/acc/info?mid="+ str(uid) +"&jsonp=jsonp",\
                          timeout=3000)
    res = requests.get(
                'https://api.bilibili.com/x/relation/stat?vmid=' + str(uid) + '&jsonp=jsonp').text
    js_fans_data = json.loads(res)         
    source = json.loads(source.text)['data']
    data['name'] = source['name']
    data['sex'] = source['sex']
    data['sign'] = source['sign']
    data['level'] = source['level']
    data['birthday'] = source['birthday']
    data['vip'] = 1 if source['vip']['status'] == 1 else 0 
    data['following'] = js_fans_data['data']['following']
    data['fans'] = js_fans_data['data']['follower']
    return data

In [27]:
user_info_list=[]

In [29]:
user_list = barrages_csv['u_id'].tolist()
idx = -1
while idx < len(user_list):
    idx += 1
    
    if idx == len(user_list):
        break
        
    try:
        user_info_list.append(get_user_info(user_list[idx]))
    except:
        print("Connection refused by the server..and i is" + str(idx))
        time.sleep(5)
        idx -= 1
        continue

Connection refused by the server..and i is1199
Connection refused by the server..and i is1199
Connection refused by the server..and i is1200
Connection refused by the server..and i is1202
Connection refused by the server..and i is1204
Connection refused by the server..and i is1204
Connection refused by the server..and i is1204
Connection refused by the server..and i is1205
Connection refused by the server..and i is1205
Connection refused by the server..and i is3482
Connection refused by the server..and i is3496
Connection refused by the server..and i is3661
Connection refused by the server..and i is4445
Connection refused by the server..and i is4533
Connection refused by the server..and i is5050
Connection refused by the server..and i is5562
Connection refused by the server..and i is6085
Connection refused by the server..and i is6730
Connection refused by the server..and i is7617
Connection refused by the server..and i is7617
Connection refused by the server..and i is7620
Connection re

In [34]:
user_info = pd.DataFrame(data=user_info_list).drop_duplicates('u_id')